## Function Definition

In [ ]:
def dwi_to_b0(brain_path,dwi_name,b_prefix,output_name,b_output_name):
    '''
    dwi_to_b0(brain_path=brain_path,dwi_name='dwi_topuped',b_prefix='dwi',output_name='dwi_to_b0',b_output_name)
    '''
    import os
    import numpy as np
    from tqdm import tqdm
    import nibabel as nib
    from dipy.io.image import load_nifti,save_nifti
    from dipy.io import read_bvals_bvecs
    from dipy.core.gradients import gradient_table
    from dipy.core.gradients import reorient_bvecs
    from dipy.align import register_series, center_of_mass,translation,rigid
    from dipy.align._public import affine,register_dwi_series,syn_registration,register_dwi_to_template
    brain_path = brain_path
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        data_prefix = dwi_name
        b_prefix = b_prefix
        dwi_file = os.path.join(Study_folder,data_prefix+'.nii') # 4D diffusion data file name
        bval = os.path.join(Study_folder,b_prefix+'.bval') # bval file name
        bvec = os.path.join(Study_folder,b_prefix+'.bvec') # bvec file name
        #------------
        bvals, bvecs = read_bvals_bvecs(bval,bvec)
        gtab = gradient_table(bvals, bvecs,atol=0.15,b0_threshold=5)
        img_data,img_affine,img = load_nifti(dwi_file,return_img=True)
        xformed,affines = register_dwi_series(img_data,gtab,affine=img_affine,b0_ref=0,
                                              pipeline= [center_of_mass, translation,rigid,affine])#,syn_registration
        gtab = gradient_table(bvals, bvecs,atol=0.15,b0_threshold=5)
        affines1 = affines.swapaxes(1,2).swapaxes(0,1)[~gtab.b0s_mask]
        gtab_new = reorient_bvecs(gtab, affines1, atol=0.15)
        np.savetxt(b_output_name+'.bval',gtab_new.bvals[np.newaxis,:],fmt='%1.5f')
        np.savetxt(b_output_name+'.bvec',gtab_new.bvecs.T,fmt='%1.5f')
        # save_nifti_origin(input=xformed.get_data(),output_name=output_name,type_output=np.int16,
        #                   zooms=img.header.get_zooms()[:4])
        save_nifti(output_name, xformed.get_data(), img_affine,hdr=img.header)

In [ ]:
# def dwi_to_t2(brain_path,input_dwi,input_t2,b_input_name,b_output_name,registered_out):
#     '''
#     dwi_to_t2(brain_path,input_dwi='dwi_b0_resample.nii',input_t2='T2_resample.nii',b_input_name='',b_output_name='',registered_out='')
#     '''
#     import os
#     import numpy as np
#     from tqdm import tqdm
#     import nibabel as nib
#     from dipy.segment.mask import applymask
#     from dipy.align.imaffine import (transform_centers_of_mass,
#                                      AffineMap,
#                                      MutualInformationMetric)
#     from dipy.io.image import load_nifti,save_nifti
#     from dipy.align import affine_registration, register_dwi_to_template,read_mapping,write_mapping,resample
#     from dipy.io.gradients import read_bvals_bvecs
#     from dipy.core.gradients import gradient_table,reorient_bvecs
#     from dipy.align.metrics import CCMetric
#     from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
#     from dipy.viz import regtools
#     for Study_folder in tqdm(brain_path):
#         os.chdir(Study_folder)
#         dwi_file = os.path.join(Study_folder,input_dwi) # 4D diffusion data file name
#         img_data,moving_affine,img = load_nifti(dwi_file,return_img=True)
#         moving = img_data[:,:,:]
#         moving_grid2world = moving_affine

#         t2_file = os.path.join(Study_folder,input_t2) # 4D diffusion data file name
#         static,static_affine,img_t2 = load_nifti(t2_file,return_img=True)
#         static_grid2world = static_affine

#         # 
#         identity = np.eye(4)
#         affine_map = AffineMap(identity,
#                                static.shape, static_grid2world,
#                                moving.shape, moving_grid2world)
#         resampled = affine_map.transform(moving)
#         print('Original file without registration')
#         regtools.overlay_slices(static, resampled, None, 2,
#                                 "T2", "b0", "1without_registration.png",dpi=500)
#         ###############
#         c_of_mass = transform_centers_of_mass(static, static_grid2world,
#                                               moving, moving_grid2world)
#         transformed = c_of_mass.transform(moving)
#         print('Image after center of mass transform')
#         regtools.overlay_slices(static, transformed, None, 2,
#                                 "Static", "Transformed", "2transformed_com_2.png",dpi=500)
#         ###############
#         #affine parameter
#         nbins = 32
#         sampling_prop = None
#         metric = MutualInformationMetric(nbins, sampling_prop)
#         level_iters = [1000, 100, 50]
#         sigmas = [3.0, 1.0, 0.0]
#         factors = [4, 2, 1]
#         pipeline = ["center_of_mass", "translation", "rigid", "affine"]#["affine"]#
#         #affine reg
#         xformed_img, reg_affine = affine_registration(
#                                         moving,
#                                         static,
#                                         moving_affine=moving_affine,
#                                         static_affine=static_affine,
#                                         nbins=32,
#                                         metric='MI',
#                                         pipeline=pipeline,
#                                         level_iters=level_iters,
#                                         sigmas=sigmas,
#                                         factors=factors)
#         print('Image after rigid, affine registration')
#         regtools.overlay_slices(static, xformed_img, None, 2, 'Static',
#                             'Warped moving', 'warped_moving.png')
#         ####################
#         # To save roated b table
#         bval = os.path.join(Study_folder,b_input_name+'.bval') # bval file name
#         bvec = os.path.join(Study_folder,b_input_name+'.bvec') # bvec file name
#         #------------
#         bvals, bvecs = read_bvals_bvecs(bval,bvec)
#         gtab = gradient_table(bvals, bvecs,atol=0.15,b0_threshold=5)
#         affines0 = np.zeros([len(bvals),4,4])#affines.swapaxes(1,2).swapaxes(0,1)[~gtab.b0s_mask]
#         for i_affine in np.arange(len(bvals)):
#             affines0[i_affine,:,:] = reg_affine
#         affines1 = affines0[~gtab.b0s_mask]
#         gtab_new = reorient_bvecs(gtab, affines1, atol=0.15)
#         np.savetxt(b_output_name+'.bval',gtab_new.bvals[np.newaxis,:],fmt='%1.5f')
#         np.savetxt(b_output_name+'.bvec',gtab_new.bvecs.T,fmt='%1.5f')
#         ####################
#         prealign = reg_affine
#         metric = CCMetric(3)
#         level_iters = [100, 60, 20]#[10,10,5]#[80, 50, 25]#[3,2,1]#
#         sdr = SymmetricDiffeomorphicRegistration(metric, level_iters)
#         mapping = sdr.optimize(static, moving, static_affine, moving_affine, prealign)
#         warped_moving = mapping.transform(moving)
#         save_nifti('dwi_b0_registered5.nii',warped_moving,static_affine,img_t2.header)
#         np.savetxt('prealign_b0.txt',mapping.prealign)
#         np.savetxt('prealign_inv_b0.txt',mapping.prealign_inv)

#         write_mapping(mapping,'syn_mapping.nii')
#         print('Image after sym registration')
#         regtools.overlay_slices(static, warped_moving, None, 2, 'Static',
#                             'Warped moving', 'warped_moving.png')
#         warped_static = mapping.transform_inverse(static)
#         regtools.overlay_slices(warped_static, moving, None, 2, 'Warped static',
#                                 'Moving', '6warped_static.png',dpi=500)
#         #################
#         affinemap = AffineMap(prealign, domain_grid_shape=moving.shape, domain_grid2world=moving_affine,
#                     codomain_grid_shape=static, codomain_grid2world=static_affine)
#         affine_img = resample(moving, static, moving_affine=moving_affine, static_affine=static_affine,between_affine=prealign)
#         save_nifti('b0_to_t2_affine.nii',affine_img.get_fdata(),affine_img.affine,img_t2.header)
#         # affine reverse transform
#         affine_inv = affinemap.transform_inverse(static, image_grid2world=static_affine, sampling_grid_shape=moving.shape,
#                             sampling_grid2world=None, resample_only=False)
#         save_nifti('b0_to_t2_affine_inv.nii',affine_inv,moving_affine,img.header)
#         #
#         mapping_file = 'syn_mapping.nii'
#         mapping = read_mapping(mapping_file,domain_img=input_dwi,codomain_img=input_t2)#prealign
#         x_transformed = mapping.transform(affine_img.get_fdata())
#         save_nifti('b0_to_t2_syn.nii',x_transformed,static_affine,img_t2.header)
#         #
#         x_transformed_inv = mapping.transform_inverse(static,image_world2grid=static_affine)
#         x_affine_inv = affinemap.transform_inverse(x_transformed_inv, image_grid2world=static_affine, sampling_grid_shape=moving.shape,
#                             sampling_grid2world=None, resample_only=False)
#         save_nifti('b0_to_t2_sym_inv.nii',x_affine_inv,moving_affine,img_t2.header)
#         ###################
#         print('Done!')

In [ ]:
def dwi_to_t2(brain_path,input_dwi,input_t2,b_input_name,b_output_name,registered_out):
    '''
    dwi_to_t2(brain_path,input_dwi='dwi_b0_resample.nii',input_t2='T2_resample.nii',b_input_name='',b_output_name='',registered_out='')
    '''
    import os
    import numpy as np
    from tqdm import tqdm
    import nibabel as nib
    from dipy.segment.mask import applymask
    from dipy.align.imaffine import (transform_centers_of_mass,
                                     AffineMap,
                                     MutualInformationMetric)
    from dipy.io.image import load_nifti,save_nifti
    from dipy.align import affine_registration, register_dwi_to_template,read_mapping,write_mapping,resample
    from dipy.io.gradients import read_bvals_bvecs
    from dipy.core.gradients import gradient_table,reorient_bvecs
    from dipy.align.metrics import CCMetric
    from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
    from dipy.viz import regtools
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        dwi_file = os.path.join(Study_folder,input_dwi) # 4D diffusion data file name
        img_data,moving_affine,img = load_nifti(dwi_file,return_img=True)
        moving = img_data[:,:,:]
        moving_affine = np.eye(4)
        moving_grid2world = moving_affine
        

        moving_mask,mask_aff=load_nifti('dwi_b0_brain_mask.nii.gz',return_img=False)
        # static_mask = np.pad(static_mask, [(pad_by1, pad_by1), (pad_by2, pad_by2), (0, 0)],
        #         mode='constant', constant_values=0)

        t2_file = os.path.join(Study_folder,input_t2) # 4D diffusion data file name
        static,static_affine,img_t2 = load_nifti(t2_file,return_img=True)
        aff_saving = static_affine-1.0

        static_affine = np.eye(4)
        static_grid2world = static_affine

        static_mask,mask_aff=load_nifti('T2w_brain_mask.nii.gz',return_img=False)
        #padding to make registration working well
        pad_by1 = (moving.shape[0]-static.shape[0])//2
        pad_by2 = (moving.shape[1]-static.shape[1])//2
        if (pad_by1>=0) and (pad_by2>=0):
                moving = moving[pad_by1:-pad_by1,pad_by2:-pad_by2,:]
        elif (pad_by1>=0) and (pad_by2<0):
                moving = moving[pad_by1:-pad_by1,:,:]
                moving = np.pad(moving, [(0, 0), (-pad_by2, -pad_by2), (0, 0)],
                                mode='constant', constant_values=0)
        elif (pad_by1<0) and (pad_by2>=0):
                moving = moving[:,pad_by2:-pad_by2,:]
                moving = np.pad(moving, [(-pad_by1, -pad_by1),(0, 0), (0, 0)],
                                mode='constant', constant_values=0)
        elif (pad_by1<0) and (pad_by2<0):
                moving = np.pad(moving, [(-pad_by1, -pad_by1),(-pad_by2, -pad_by2), (0, 0)],
                                mode='constant', constant_values=0)
        # 
        # 
        identity = np.eye(4)
        affine_map = AffineMap(identity,
                               static.shape, static_grid2world,
                               moving.shape, moving_grid2world)
        resampled = affine_map.transform(moving)
        print('Original file without registration')
        regtools.overlay_slices(static, resampled, None, 2,
                                "T2", "b0", "1without_registration.png",dpi=500)
        c_of_mass = transform_centers_of_mass(static, static_grid2world,
                                              moving, moving_grid2world)
        transformed = c_of_mass.transform(moving)
        print('Image after center of mass transform')
        regtools.overlay_slices(static, transformed, None, 2,
                                "Static", "Transformed", "2transformed_com_2.png",dpi=500)
        #affine parameter
        nbins = 32
        sampling_prop = None
        metric = MutualInformationMetric(nbins, sampling_prop)
        level_iters = [10000, 1000, 500]
        sigmas = [3.0, 1.0, 0.0]
        factors = [4, 2, 1]
        pipeline = ["center_of_mass","translation", "rigid", "affine"]#["affine"]#
        #affine reg
        xformed_img, reg_affine = affine_registration(
                                        moving,
                                        static,
                                        moving_affine=moving_affine,
                                        static_affine=static_affine,
                                        nbins=32,
                                        metric='MI',
                                        pipeline=pipeline,
                                        level_iters=level_iters,
                                        sigmas=sigmas,
                                        factors=factors,
                                        # static_mask=static_mask,
                                        # moving_mask=moving_mask,
                                        )
        regtools.overlay_slices(static, xformed_img, None, 2, 'Static',
                            'Warped moving', 'warped_moving.png')

        prealign = reg_affine

        metric = CCMetric(3)
        level_iters = [50, 30, 10]#[10,10,5]#[80, 50, 25]#[3,2,1]#
        sdr = SymmetricDiffeomorphicRegistration(metric, level_iters)
        mapping = sdr.optimize(static, moving, static_affine, moving_affine, prealign)
        warped_moving = mapping.transform(moving)
        #name
        name = 'test'
        np.savetxt(os.path.join(Study_folder,name+'_flash3d_reg_affine.txt'),reg_affine)
        write_mapping(mapping,os.path.join(Study_folder,name+'_flash3d_mapping.nii'))
        regtools.overlay_slices(static, warped_moving, None, 2, 'Static',
                            'Warped moving', 'warped_moving.png')
        save_nifti('b0_to_t2_aff_sym.nii',warped_moving,aff_saving+1,img_t2.header)

In [ ]:
# def dwi_to_t2(brain_path,dwi_all_file,input_dwi,input_t2,b_input_name,b_output_name,registered_out):
#     '''
#     dwi_to_t2(brain_path,dwi_all_file=,input_dwi='dwi_b0_resample.nii',input_t2='T2_resample.nii',b_input_name='',b_output_name='',registered_out='')
#     '''
#     import os
#     import numpy as np
#     from tqdm import tqdm
#     import nibabel as nib
#     from dipy.core.gradients import gradient_table
#     from dipy.data import get_fnames
#     from dipy.io.gradients import read_bvals_bvecs
#     from dipy.core.gradients import reorient_bvecs
#     from dipy.segment.mask import applymask
#     from dipy.io.image import load_nifti,save_nifti
#     from dipy.align._public import write_mapping,read_mapping
#     from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
#     from dipy.align.imwarp import DiffeomorphicMap
#     from dipy.align.metrics import CCMetric
#     from dipy.viz import regtools
#     from os.path import join as pjoin
#     from dipy.align.imaffine import (transform_centers_of_mass,
#                                      AffineMap,
#                                      MutualInformationMetric,
#                                      AffineRegistration)
#     from dipy.align.transforms import (TranslationTransform3D,
#                                        RigidTransform3D,
#                                        AffineTransform3D)
#     for Study_folder in tqdm(brain_path):
#         os.chdir(Study_folder)
#         dwi_file = os.path.join(Study_folder,input_dwi) # 4D diffusion data file name
#         img_data,moving_affine,img = load_nifti(dwi_file,return_img=True)
#         moving = img_data
#         moving_grid2world = moving_affine

#         t2_file = os.path.join(Study_folder,input_t2) # 4D diffusion data file name
#         static,static_affine,t2 = load_nifti(t2_file,return_img=True)
#         static_grid2world = static_affine

#         # 
#         identity = np.eye(4)
#         affine_map = AffineMap(identity,
#                                static.shape, static_grid2world,
#                                moving.shape, moving_grid2world)
#         resampled = affine_map.transform(moving)
#         regtools.overlay_slices(static, resampled, None, 2,
#                                 "T2", "b0", "1without_registration.png",dpi=500)
#         ###############
#         c_of_mass = transform_centers_of_mass(static, static_grid2world,
#                                               moving, moving_grid2world)
#         transformed = c_of_mass.transform(moving)
#         regtools.overlay_slices(static, transformed, None, 0,
#                                 "Static", "Transformed", "2transformed_com_0.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 1,
#                                 "Static", "Transformed", "2transformed_com_1.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 2,
#                                 "Static", "Transformed", "2transformed_com_2.png",dpi=500)
#         ###############
#         nbins = 32
#         sampling_prop = None
#         metric = MutualInformationMetric(nbins, sampling_prop)
#         level_iters = [10000, 1000, 100]
#         sigmas = [3.0, 1.0, 0.0]
#         factors = [4, 2, 1]
#         affreg = AffineRegistration(metric=metric,
#                                     level_iters=level_iters,
#                                     sigmas=sigmas,
#                                     factors=factors)
#         transform = TranslationTransform3D()
#         params0 = None
#         starting_affine = c_of_mass.affine
#         translation = affreg.optimize(static, moving, transform, params0,
#                                       static_grid2world, moving_grid2world,
#                                       starting_affine=starting_affine)
#         transformed = translation.transform(moving)
#         regtools.overlay_slices(static, transformed, None, 0,
#                                 "Static", "Transformed", "3transformed_trans_0.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 1,
#                                 "Static", "Transformed", "3transformed_trans_1.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 2,
#                                 "Static", "Transformed", "3transformed_trans_2.png",dpi=500)
#         #################
#         transform = RigidTransform3D()
#         params0 = None
#         starting_affine = translation.affine
#         rigid = affreg.optimize(static, moving, transform, params0,
#                                 static_grid2world, moving_grid2world,
#                                 starting_affine=starting_affine)
#         transformed = rigid.transform(moving)
#         nib.save(nib.Nifti1Image(transformed,np.eye(4)),'dwi_b0_registered3.nii')
#         regtools.overlay_slices(static, transformed, None, 0,
#                                 "Static", "Transformed", "4transformed_rigid_0.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 1,
#                                 "Static", "Transformed", "4transformed_rigid_1.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 2,
#                                 "Static", "Transformed", "4transformed_rigid_2.png",dpi=500)
#         ###################
#         transform = AffineTransform3D()
#         params0 = None
#         starting_affine = rigid.affine
#         affine = affreg.optimize(static, moving, transform, params0,
#                                  static_grid2world, moving_grid2world,
#                                  starting_affine=starting_affine)
#         transformed = affine.transform(moving)
#         nib.save(nib.Nifti1Image(transformed,np.eye(4)),'dwi_b0_registered4.nii')
#         regtools.overlay_slices(static, transformed, None, 0,
#                                 "Static", "Transformed", "4transformed_affine_0.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 1,
#                                 "Static", "Transformed", "4transformed_affine_1.png",dpi=500)
#         regtools.overlay_slices(static, transformed, None, 2,
#                                 "Static", "Transformed", "4transformed_affine_2.png",dpi=500)
#         ####################
#         # To save roated b table
        
#         bval = os.path.join(Study_folder,b_input_name+'.bval') # bval file name
#         bvec = os.path.join(Study_folder,b_input_name+'.bvec') # bvec file name
#         #------------
#         bvals, bvecs = read_bvals_bvecs(bval,bvec)
#         gtab = gradient_table(bvals, bvecs,atol=0.15,b0_threshold=5)
#         affines0 = np.zeros([len(bvals),4,4])#affines.swapaxes(1,2).swapaxes(0,1)[~gtab.b0s_mask]
#         for i_affine in np.arange(len(bvals)):
#             affines0[i_affine,:,:] = affine.affine
#         affines1 = affines0[~gtab.b0s_mask]
#         gtab_new = reorient_bvecs(gtab, affines1, atol=0.15)
#         np.savetxt(b_output_name+'.bval',gtab_new.bvals[:,np.newaxis].T,fmt='%1.3f')
#         np.savetxt(b_output_name+'.bvec',gtab_new.bvecs.T,fmt='%1.3f')
#         ####################
        
#         metric = CCMetric(3)
#         level_iters = [100, 100, 25]
#         sdr = SymmetricDiffeomorphicRegistration(metric, level_iters)
#         pre_align = affine.affine
#         mapping = sdr.optimize(static, moving, static_grid2world, moving_grid2world, pre_align)
#         warped_moving = mapping.transform(moving)
#         nib.save(nib.Nifti1Image(warped_moving,np.eye(4)),'dwi_b0_registered5.nii')

#         from dipy.align._public import write_mapping,read_mapping
#         write_mapping(mapping,'syn_mapping.nii')
#         np.savetxt('prealign_b0.txt',mapping.prealign)
#         prealign_b0 = np.loadtxt('prealign_b0.txt')
#         np.savetxt('prealign_inv_b0.txt',mapping.prealign_inv)
#         prealign_inv_b0 = np.loadtxt('prealign_inv_b0.txt')

#         regtools.overlay_slices(static, warped_moving, None, 1, 'Static',
#                                 'Warped moving', '5warped_moving.png',dpi=500)
#         regtools.overlay_slices(static, warped_moving, None, 2, 'Static',
#                                 'Warped moving', '5warped_moving.png',dpi=500)
#         warped_static = mapping.transform_inverse(static)
#         regtools.overlay_slices(warped_static, moving, None, 1, 'Warped static',
#                                 'Moving', '6warped_static.png',dpi=500)
#         regtools.overlay_slices(warped_static, moving, None, 2, 'Warped static',
#                                 'Moving', '6warped_static.png',dpi=500)
#         #################
#         # read and write mapping test
#         prealign_b0 = np.loadtxt('prealign_b0.txt')
#         mapping_0 = read_mapping(disp='syn_mapping.nii',domain_img=input_dwi,
#                                  codomain_img=input_t2,prealign=prealign_b0)
#         test_data,test_affine,test_img = load_nifti(input_dwi,return_img=True)

#         test_trans = mapping_0.transform(test_data[:,:,:])
#         new_shapes = test_trans.shape
#         img_new = nib.Nifti1Image(test_trans.astype(np.int16), np.eye(4))
#         new_zooms=[1,1,1]
#         img_new.header.set_zooms(new_zooms)
#         img_new.header['intent_code'] = 1005
#         img_new.header['sform_code'] = 1
#         img_new.header['qform_code'] = 1
#         img_new.header['dim'][0] = 3#[4,128,128,14,6,1,0,0]
#         img_new.header['dim'][1] = new_shapes[0]
#         img_new.header['dim'][2] = new_shapes[1]
#         img_new.header['dim'][3] = new_shapes[2]
#         save_nifti('dwi_b0_registered_test',test_trans.astype(np.int16),affine=np.eye(4),hdr=img_new.header)
#         ###################
#         # transform of dwi data
#         mapping_0 = read_mapping(disp='syn_mapping.nii',domain_img=input_dwi,
#                                  codomain_img=input_t2,prealign=prealign_b0)
#         test_data,test_affine,test_img = load_nifti(dwi_all_file,return_img=True)
#         test_trans = np.zeros(mapping.transform(test_data[:,:,:,0]).shape+(test_data.shape[-1],))
#         for i_vol in np.arange(test_data.shape[-1]):
#             test_trans[:,:,:,i_vol] = mapping.transform(test_data[:,:,:,i_vol])

#         new_shapes = test_trans.shape
#         img_new = nib.Nifti1Image(test_trans.astype(np.int16), np.eye(4))
#         new_zooms=[1,1,1,8]
#         img_new.header.set_zooms(new_zooms)
#         img_new.header['intent_code'] = 1005
#         img_new.header['sform_code'] = 1
#         img_new.header['qform_code'] = 1
#         img_new.header['dim'][0] = 4#[4,128,128,14,6,1,0,0]
#         img_new.header['dim'][1] = new_shapes[0]
#         img_new.header['dim'][2] = new_shapes[1]
#         img_new.header['dim'][3] = new_shapes[2]
#         img_new.header['dim'][4] = new_shapes[3]
#         save_nifti(registered_out,test_trans.astype(np.int16),affine=np.eye(4),hdr=img_new.header)
#         ####################
#         print('Done!')

In [ ]:
def dwi_resample(brain_path,data_in,data_out,zooms_new):
    '''
    '''
    import os
    import numpy as np
    from dipy.io.image import load_nifti, save_nifti
    from dipy.segment.mask import applymask
    from dipy.io import read_bvals_bvecs
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    from dipy.reconst.dti import TensorFit
    from dipy.reconst.dti import color_fa
    from tqdm.notebook import tqdm
    from dipy.viz import regtools
    import dipy.denoise.noise_estimate as ne
    import nibabel as nib
    from dipy.align.reslice import reslice
    brain_path = brain_path
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        data_prefix = data_in
        dwi_file = os.path.join(Study_folder,data_prefix) # 4D diffusion data file name
        img_data,img_affine,img = load_nifti(dwi_file,return_img=True)
        zooms_old = img.header.get_zooms()[:3]
        new_zooms = zooms_new[0:3]
        if len(img_data.shape) == 4:
            test,affine_test = reslice(img_data[:,:,:,0], img_affine, zooms_old, new_zooms)
            data_new = np.zeros(test.shape+(img_data.shape[-1],))
            for i_slice in np.arange(img_data.shape[-1]):
                data_new[:,:,:,i_slice], affine_new = reslice(img_data[:,:,:,i_slice], img_affine, zooms_old, new_zooms)
        elif len(img_data.shape) == 3:
            data_new,affine_test = reslice(img_data, img_affine, zooms_old, new_zooms)
        else:
            print('input file error!')
        # img_resample = nib.Nifti1Image(data_new.astype(np.int16), np.eye(4))
        # save_nifti_origin(input=img_resample.get_data(),output_name=data_out,type_output=np.int16,
        #                   zooms=zooms_new)
        img.header.set_zooms(zooms_new)
        save_nifti(data_out, data_new, img_affine,hdr=img.header)

In [ ]:
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/Registration_test']
dwi_to_b0(brain_path=brain_path,dwi_name='dwi_topuped',b_prefix='dwi_mb0',
          output_name='dwi_to_b0',b_output_name='dwi_to_b0')

In [ ]:
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/Registration_test']
dwi_resample(brain_path,data_in='dwi_to_b0.nii',data_out='dwi_resampled.nii',zooms_new=[1.0,1.0,1.0,8])
dwi_resample(brain_path,data_in='T2w.nii',data_out='T2_resampled.nii',zooms_new=[1.0,1.0,1.0])

In [ ]:
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/Registration_test']
dwi_to_t2(brain_path,'dwi_b0_brain.nii',input_t2='T2w_brain.nii',
          b_input_name='dwi_to_b0',b_output_name='dwi_registered',registered_out='dwi_registered')
# dwi_to_t2(brain_path,dwi_all_file='dwi_to_b0.nii',input_dwi='dwi_b0_brain.nii',input_t2='T2w_brain.nii',b_input_name='dwi_to_b0',b_output_name='dwi_to_b0_registered',registered_out='dwi_to_b0_registered')

In [ ]:
import nibabel as nib
import os
import numpy as np
from dipy.io.image import load_nifti, save_nifti
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/Registration_test']

img = nib.load(os.path.join(brain_path[0],'b0_to_t2_aff_sym.nii'))
data = img.get_fdata()
pad_by1 = 60
pad_by2 = 10
# data = np.pad(data, [(pad_by1, pad_by1), (pad_by2, pad_by2), (0, 0)],
#         mode='constant', constant_values=0)
print(data.shape)
data = data[pad_by1:-pad_by1,pad_by2:-pad_by2,:]
print(data.shape)
# img.header['pixdim'][1:4]  = data.shape
save_nifti('b0_to_t2_aff_sym2.nii',data,img.affine,img.header)